<a href="https://colab.research.google.com/github/Gressling/S88-light/blob/main/sequence/Example/run_example_sequence_and_unit_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# go

In [ ]:
pip install bonobo lxml

In [ ]:
# Dirty bugfix due to python 3.10
import collections.abc
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

In [ ]:
import requests
import bonobo
from lxml import etree

## Unit Operation implementations (example)

In [ ]:
class Reaction:
    def __init__(self, parameters):
        self.parameters = parameters
    def execute(self):
        print("Executing Reaction with parameters:", self.parameters)


class AddOnce:
    def __init__(self, parameters):
        self.parameters = parameters
    def execute(self):
        print("Executing AddOnce with parameters:", self.parameters)


class Dry:
    def __init__(self, parameters):
        self.parameters = parameters

    def execute(self):
        print("Executing Dry with parameters:", self.parameters)

## Sequence runner

In [ ]:
def parse_sequence(unit_operation):
    for child in unit_operation.getchildren():
        operation_name = child.tag
        print(f"Processing {operation_name}...")

        parameters = {param.tag: param.text for param in child.xpath('Parameters/*')}

        if operation_name == "Reaction":
            operation = Reaction(parameters)
        elif operation_name == "AddOnce":
            operation = AddOnce(parameters)
        elif operation_name == "Dry":
            operation = Dry(parameters)
        else:
            print(f"Warning: Operation {operation_name} not recognized.")
            continue

        operation.execute()

## Read a real sequence and read unit operation parameters from a XML

In [ ]:
def reader():
    XML_URL = "https://raw.githubusercontent.com/Gressling/S88-NG/main/sequence/Example/3UnitOperations.xml"
    response = requests.get(XML_URL)
    response.raise_for_status()

    root = etree.fromstring(response.content)
    for unit_operation in root.xpath('//UnitOperation'):
        yield unit_operation

## run

In [ ]:
def get_graph(**options):
    graph = bonobo.Graph()
    graph.add_chain(reader, parse_sequence)
    return graph

In [ ]:
if __name__ == '__main__':
    print("Starting process...")
    bonobo.run(get_graph())
    print("Process completed.")

Starting process...
Processing Reaction...
Executing Reaction with parameters: {'Reactants': 'Names or Chemical Formulas of Reactants', 'Stoichiometry': 'Stoichiometry and Balanced Chemical Equation', 'ReactionTemperature': 'Temperature value (°C)', 'ReactionTime': 'Duration of Reaction (hours or days)', 'Catalysts': 'List of Catalysts or Enzymes (if any)', 'Solvent': 'Type of Solvent Used', 'ReactionPressure': 'Pressure value (optional, if applicable)', 'pH': 'pH value (optional, if applicable)', 'MixingOrAgitation': 'Level of Mixing or Agitation', 'SafetyMeasures': 'Precautions during Reaction', 'YieldAndProductivity': 'Desired Yield and Productivity', 'ProductIsolationAndPurification': 'Methods of Isolation and Purification', 'ReactionMonitoring': 'Techniques for Monitoring the Reaction', 'ReactionConditions': 'Any Other Specific Conditions or Parameters'}
Processing AddOnce...
Executing AddOnce with parameters: {'Reactant': '...', 'Amount': '...', 'Timing': '...', 'RateOfAddition':